In [1]:

from IngestDrugSynonyms import IngestDrugSynonyms

In [2]:
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/ben/Projects/Graphs4GoodHackathon/ProjectDomino/.Domino/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
drugSynonym = IngestDrugSynonyms()

In [6]:
drugSynonym.scrapeData()

2020-04-07 15:51:58,259 - > 351 studies found by 'covid-19' keyword
2020-04-07 15:52:11,769 - > 351 studies found by 'SARS-CoV-2' keyword
2020-04-07 15:52:25,084 - > 301 studies found by 'coronavirus' keyword


In [7]:
drugSynonym.filterData()

In [8]:
drugSynonym.internationalstudies.head()

,TrialID,Last Refreshed on,Public title,Scientific title,Acronym,Primary sponsor,Date registration,Date registration3,Export date,Source Register,web address,Recruitment Status,other records,Inclusion agemin,Inclusion agemax,Inclusion gender,Date enrollement,Target size,Study type,Study design,Phase,Countries,Contact Firstname,Contact Lastname,Contact Address,Contact Email,Contact Tel,Contact Affiliation,Inclusion Criteria,Exclusion Criteria,Condition,Intervention,Primary outcome,results date posted,results date completed,results url link,Retrospective flag,Bridging flag truefalse,Bridged type,results yes no
0,ChiCTR2000029953,43878.0,Construction and Analysis of Prognostic Predictive Model of Novel Coronavirus Pneumonia (COVID-19),Construction and Analysis of Prognostic Predictive Model of Novel Coronavirus Pneumonia (COVID-19),NaN,Zhongnan Hospital of Wuhan University,43878.0,20200217.0,43834.660579,ChiCTR,http://www.chictr.org.cn/showproj.aspx?proj=49217,Not Recruiting,No,NaN,NaN,Both,43862.0,survival group:200;died:200;,Observational study,Factorial,NaN,China,Yan Zhao,NaN,"169 Donghu Road, Wuchang District, Wuhan, Hubei, China",doctoryanzhao@whu.edu.cn,+86 13995577963,Emergency Department of Zhongnan Hospital of Wuhan University,"Inclusion criteria: 2019-nCoV-infected pneumonia diagnosed patients, the diagnostic criteria refer to ""guideline of the diagnose and treatment of 2019-nCoV (trial)"" published by National Health Committees of Peoples' Republic Country ""","Exclusion criteria: Patients diagnosed with pneumonia of age <15 years diagnosed according to""guideline of the diagnose and treatment of 2019-nCoV (tested)"" published by National Health Committees of Peoples' Republic Country; Suspected patient with vira",2019-nCoV Pneumonia,survival group:none;died:none;,duration of in hospital;in hospital mortality;the 28s day' mortality after admission;duration of ICU stay;,NaN,NaN,NaN,No,0,,NaN
1,ChiCTR2000029935,43878.0,A Single-arm Clinical Trial for Chloroquine Phosphate in the treatment of Novel Coronavirus Pneumonia 2019 (COVID-19),A Single-arm Clinical Trial for Chloroquine Phosphate in the treatment of Novel Coronavirus Pneumonia 2019 (COVID-19),NaN,"HwaMei Hospital, University of Chinese Academy of Sciences",43877.0,20200216.0,43834.660579,ChiCTR,http://www.chictr.org.cn/showproj.aspx?proj=49607,Recruiting,No,NaN,NaN,Both,43867.0,Case series:100;,Interventional study,Single arm,NaN,China,Ting Cai,NaN,"41 Xibei Street, Ningbo, Zhejiang, China",caiting@ucas.ac.cn,+86 13738498188,"HwaMei Hospital, University of Chinese Academy of Sciences","Inclusion criteria: Patients aged 18 or older, and meet the diagnostic criteria of Diagnosis and Treatment Scheme of Novel Coronavirus Infected Pneumonia published by the National Health Commission.\r<br>Criteria for diagnosis (meet all the following crite",Exclusion criteria: 1. Female patients during pregnancy; \r<br>2. Patients with known allergy to chloroquine; \r<br>3. Patients with haematological diseases; \r<br>4. Patients with chronic liver or kidney diseases at the end stage; \r<br>5. Patients with arrh,Novel Coronavirus Pneumonia (COVID-19),Case series:Treated with conventional treatment combined with Chloroquine Phosphate;,Length of hospital stay;,NaN,NaN,NaN,No,0,,NaN
2,ChiCTR2000029850,43878.0,Study on convalescent plasma treatment for severe patients with novel coronavirus pneumonia (COVID-19),Effecacy and safty of convalescent plasma treatment for severe patients with novel coronavirus pneumonia (COVID-19): a prospective cohort study,NaN,The First Affiliated Hospital of Zhejiang University School of Medicine,43876.0,20200215.0,43834.660579,ChiCTR,http://www.chictr.org.cn/showproj.aspx?proj=49533,Recruiting,No,16.0,99.0,Male,43876.0,experimental group:10;control group:10;,Interventional study,Non randomized control,0.0,China,Xiaowei Xu,NaN,"79 Qingchun Road, Shangcheng District, Hangzhou, Zhejiang, China",xxw69@126.com,+86 13605708066,"The First Affiliated Hospital o

In [9]:
drugSynonym.drug_vocab_reduced.head()

,Common name,Synonyms
0,Lepirudin,Hirudin variant-1 | Lepirudin recombinant
1,Cetuximab,"Cetuximab | Cétuximab | Cetuximabum | Immunoglobulin G 1 (human-mouse monoclonal C 225 gamma 1 - chain anti-human epidermal growt factor receptor), disulfide wit human-mouse monoclonal C 225 kappa - chain, dimer"
2,Dornase alfa,"Deoxyribonuclease (human clone 18-1 protein moiety) | Dornasa alfa | Dornase alfa, recombinant | Dornase alpha | Recombinant deoxyribonuclease (DNAse)"
3,Denileukin diftitox,Denileukin | Interleukin-2/diptheria toxin fusion protein
4,Etanercept,Etanercept | etanercept-szzs | etanercept-ykro | Recombinant human TNF | rhu TNFR:Fc | rhu-TNFR:Fc | TNFR-Immunoadhesin


In [10]:
len(drugSynonym.drug_vocab)

13475

In [11]:
inter = drugSynonym.internationalstudies